En este notebook, se transforman los archivos de Yelp y Google a formato parquet.


Se asume que existe la siguente estructura de directorios:
```
├── data
│   ├── Google
│   │   ├── metadata-sitios
│   │   └── reviews-estados
│   └── Yelp
├── ...
└── ...
```

In [2]:
import os
from glob import glob
import json
import dask.dataframe as dd
import pandas as pd
import pyarrow as pa
from pyarrow import parquet as pq

In [2]:
# Opción que deshabilita el limite de columnas mostradas
pd.set_option('display.max_columns', None)

---
## Dataset Yelp

### `business.pkl`

In [ ]:
business_path = './data/Yelp/business.pkl'
# Leer pkl (pickle)
df = pd.read_pickle(business_path)
df.sample(2)

* Se observan columnas con nombres duplicados. Hay que unir estas columns.

In [ ]:
# Calcular la cantida de columns dividida por 2
punto_medio = len(df.columns) // 2
# Dividir el dataframe por mitad, a largo de las columns,
# utilizando el punto_medio calculado
df_1er_mitad = df.iloc[:, :punto_medio]
df_2nd_mitad = df.iloc[:, punto_medio:]
# Unir los dataframes resultantes
df_reorganizado = pd.concat([df_1er_mitad, df_2nd_mitad], axis=0)
df_reorganizado.sample(2)

In [ ]:
# Eliminar filas con todo nulos
df_reorganizado.dropna(axis=0, how='all', inplace=True)

In [ ]:
# Almacenar en formato parquet, en el mismo directorio de 'business_path'
df_reorganizado.to_parquet(business_path.replace('.pkl', '.parquet'))  # se cambia sufijo '.pkl' a '.parquet'

### `checkin.json`

In [ ]:
checkin_path = './data/Yelp/checkin.json'
# Leer JSON
df = pd.read_json(checkin_path, lines=True)
# Almacenar en formato parquet, en el mismo directorio
df.to_parquet(checkin_path.replace('.json', '.parquet'))  # se cambia sufijo '.json' a '.parquet'

### `review.json`

In [35]:
review_path = './data/Yelp/review.json'
# Crear el dir del almacen para parquets, si no existe
os.makedirs('./data/Yelp/yelp-review/', exist_ok=True)

In [36]:
def convertir_json_a_parquets(json_path, output_dir, batch_size=500_000):
    # Abrir el archivo JSON
    with open(json_path, 'r') as json_file:
        # Inicializar una lista vacía para contener los objetos JSON
        objects = []
        contador = 0
        
        # Leer el archivo línea por línea
        for line in json_file:
            # Analizar cada línea como un objeto JSON
            obj = json.loads(line)

            # Agregar el objeto a la lista
            objects.append(obj)

            # Si la lista ha alcanzado el tamaño del batch_size, escribirlo en un archivo Parquet
            if len(objects) == batch_size:
                df = pd.DataFrame(objects)
                contador += 1
                pq.write_table(pa.Table.from_pandas(df), os.path.join(output_dir, f'review_{str(contador).zfill(2)}.parquet'))
                objects = []

        # Escribir cualquier objeto restante en un archivo Parquet
        if objects:
            df = pd.DataFrame(objects)
            contador += 1
            pq.write_table(pa.Table.from_pandas(df), os.path.join(output_dir, f'review_{str(contador).zfill(2)}.parquet'))


In [ ]:
# Convertir 'review.json' a archivos parquet
convertir_json_a_parquets(review_path, './data/Yelp/review/')

### `tip.json`

In [ ]:
tip_path = './data/Yelp/tip.json'
# Leer JSON
df = pd.read_json(tip_path, lines=True)
# Almacenar en formato parquet, en el mismo directorio
df.to_parquet(tip_path.replace('.json', '.parquet'))  # se cambia sufijo '.json' a '.parquet'Merge de parquets

### `user.parquet`

Esta en formato parquet, pero es de aprox. 2.7GB. Hay que dividirlo en fragmentos manejables (250MB).

In [7]:
user_path = './data/Yelp/user.parquet'
# Crear el dir del almacen para parquets, si no existe
user_dir = './data/Yelp/yelp-user/'
os.makedirs(user_dir, exist_ok=True)

In [8]:
# Leer 'user.parquet' 
df = dd.read_parquet(user_path)
# Reparticionar la data a archivos de 250MB
df = df.repartition(partition_size='250MB')
# Almacenar en 'user_dir'
df.to_parquet(user_dir)

---
## Dataset Google

### `metadata-sitios/`

In [ ]:
# Crear lista de JSON en /metadata-sitios/
google_dir = './data/Google'
metadata_jsons = glob(f'{google_dir}/metadata-sitios/*json')
metadata_jsons

In [ ]:
## Convertir archivos JSON, en /metadata-sitios/, a parquet

# Crear dataframe donde se unen los datos extraidos de JSON
union_df = pd.DataFrame()
# Iterar por cada JSON dentro de /metadata-sitios/
for f_json in metadata_jsons:
    # Leer JSON
    df = pd.read_json(f_json, lines=True)
    # Unir a union_df
    union_df = pd.concat([union_df, df], ignore_index=True)

# Almacenar en formato parquet, en el mismo directorio
union_df.to_parquet(f'{google_dir}/metadata-sitios/metadata_sitios.parquet')

### `reviews-estados/`

In [ ]:
# Lista de directorios en /reviews-estados/
google_dir = './data/Google'
review_dirs = glob(f'{google_dir}/reviews-estados/*')
review_dirs

In [ ]:
## Convertir archivos JSON, en todo directorio dentro de /reviews-estados/, a parquet

# Iterar por cada directorio dentro de /reviews-estados/
for path_dir in review_dirs:
    # Crear lista de JSON dentro de 'path_dir'
    path_jsons = glob(f'{path_dir}/*.json')
    # Crear dataframe donde se unen los datos extraidos de JSON
    union_df = pd.DataFrame()

    # Iterar por cada JSON dentro de 'path_jsons' y almacenar su data en 'union_df'
    for f_json in path_jsons:
        # Leer JSON
        df = pd.read_json(f_json, dtype={'user_id':str}, lines=True)  # fijando 'user_id' a tipo str
        # Unir a union_df
        union_df = pd.concat([union_df, df], ignore_index=True)

    # Almacenar 'union_df' como parquet, en el path './data/Google/reviews-estados/'
    path_almacen = f'{google_dir}/reviews-estados'
    nombre_archivo = f'{path_dir.split("/")[-1].replace("-","_")}'
    union_df.to_parquet(f'{path_almacen}/{nombre_archivo}.parquet')

---
## Tranformacion a CSV, para visualizar

### Yelp

Nota: para dataset Yelp, falta `review.json`

In [38]:
review_path = './data/Yelp/review/review_01.parquet'
# Leer parquet
df = pd.read_parquet(review_path)
# Almacenar como CSV, un ejemplar de 100 filas
df.sample(100).to_csv(f'./data_csv/yelp_{review_path.split("/")[-1].replace(".parquet","")}.csv')

In [ ]:
business_path = './data/Yelp/business.parquet'
# Leer parquet
df = pd.read_parquet(business_path)
# Almacenar como CSV, un ejemplar de 100 filas
df.sample(100).to_csv(f'./data_csv/yelp_{business_path.split("/")[-1].replace(".parquet","")}.csv')

In [ ]:
checkin_path = './data/Yelp/checkin.parquet'
# Leer parquet
df = pd.read_parquet(checkin_path)
# Almacenar como CSV, un ejemplar de 100 filas
df.sample(100).to_csv(f'./data_csv/yelp_{checkin_path.split("/")[-1].replace(".parquet","")}.csv')

In [ ]:
tip_path = './data/Yelp/tip.parquet'
# Leer parquet
df = pd.read_parquet(tip_path)
# Almacenar como CSV, un ejemplar de 100 filas
df.sample(100).to_csv(f'./data_csv/yelp_{tip_path.split("/")[-1].replace(".parquet","")}.csv')

In [ ]:
user_path = './data/Yelp/user.parquet'
# Leer parquet
df = pd.read_parquet(user_path)
# Almacenar como CSV, un ejemplar de 100 filas
df.sample(100).to_csv(f'./data_csv/yelp_{user_path.split("/")[-1].replace(".parquet","")}.csv')

### Google

In [ ]:
google_dir = './data/Google'

In [ ]:
# Leer parquet
df = pd.read_parquet(f'{google_dir}/metadata-sitios/1.parquet')
# Almacenar como CSV, un ejemplar de 100 filas
df.sample(100).to_csv(f'./data_csv/google_metadata.csv')

In [ ]:
# Leer parquet
df = pd.read_parquet(f'{google_dir}/reviews-estados/review_District_of_Columbia.parquet')
# Almacenar como CSV, un ejemplar de 100 filas
df.sample(100).to_csv(f'./data_csv/google_reviews.csv')